In [ ]:
  import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
px.colors.qualitative.T10

['#4C78A8',
 '#F58518',
 '#E45756',
 '#72B7B2',
 '#54A24B',
 '#EECA3B',
 '#B279A2',
 '#FF9DA6',
 '#9D755D',
 '#BAB0AC']

In [ ]:
array = [
    [[0,1,0.1],   [0,1,0.1],    [0,1,0.1],    [0,0,0.1],       [0,0,0.1],       [0,0,0.1],],
    [[0.5,2,0.1], [0.5,3,0.1],  [0.5,3,0.1],  [0.5,0,0.1],     [0.5,0,0.1],     [0.5,0,0.1],],
    [[1,2,0.1],   [1,3,0.1],    [1,4,0.1],   [1,0,0.1],       [1,0,0.1],       [1,0,0.1],],
    [[1.5,1,0.1], [1.5,1,0.1],  [1.5,1,0.1], [1.5,0,0.1],     [1.5,0,0.1],     [1.5,0,0.1],],
    
]

array = np.stack(array,axis=1)

In [ ]:

fig = go.Figure()

c = px.colors.qualitative.T10

name = ['Full', 'Plain', '$m_{inv}^2$', '$\Delta$', '$k_T$', 'z']

for i, data in enumerate(array):
  x = data[:,0]
  mean = data[:,1]
  std = data[:,2]
  fig.add_trace(go.Scatter(x=x, y=mean+std,
                          mode='lines',
                          line=dict(color=c[i],width =0.1),
                          #fill='tonexty',
                          name='bar'))
  fig.add_trace(go.Scatter(x=x, y=mean,
                          mode='lines',
                          line=dict(color=c[i]),
                          fill='tonexty',
                          name=name[i]))
  fig.add_trace(go.Scatter(x=x, y=mean-std,
                          mode='lines',
                          line=dict(color=c[i], width =0.1),
                          fill='tonexty',
                          name='bar'))
for trace in fig['data']: 
    if(trace['name'] == 'bar'): trace['showlegend'] = False

fig.update_layout(xaxis_title='$\alpha_x$',
                  yaxis_title='AUC',
                  title={'text': '<b>Particle Transformer: y-z Boost</b>', 'font': {'size': 30}}#title='Particle Transformer: y-z Boost'
                  ,)
fig.show()

fig.write_image("y-z_test.png", width=800, height=800)


ValueError: ignored

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)



Wed Dec 14 00:53:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    56W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%pip install -U tensorflow==2.10.0
%pip install -U kaleido
%pip install -U tensorflow-addons

%pip install awkward uproot uproot_methods vector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.1 MB 5.1 kB/s 
     |████████████████████████████████| 1.7 MB 78.8 MB/s 
     |████████████████████████████████| 5.9 MB 74.7 MB/s 
     |████████████████████████████████| 438 kB 105.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully

In [ ]:

from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/MPhys\ Project/data/
import datetime
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
import tensorflow_addons as tfa
from functools import partial
import h5py
import matplotlib.pyplot as plt
import vector
import collections
import math
import string
import uproot
import awkward as ak
from kerasPMHA import KerasPMHA


Mounted at /content/drive/
/content/drive/My Drive/MPhys Project/data


In [ ]:
def histplot(history):
    hist = pd.DataFrame(history.history)
    epochs = hist.index.to_numpy() +1
    fig = make_subplots(rows=1, cols=2,subplot_titles=('Accuracy',  'Loss'))
    
    fig.add_trace(go.Scatter(mode="markers+lines", x=epochs, y=hist["Accuracy"], name = "Accuracy"), row=1, col=1)
    fig.add_trace(go.Scatter(mode="markers+lines", x=epochs, y=hist["val_Accuracy"], name = "Val accuracy"), row=1, col=1)
    
    fig.add_shape(type='line',
                x0=0,
                y0=np.max(hist["val_Accuracy"]),
                x1=np.max(epochs),
                y1=np.max(hist["val_Accuracy"]),
                line=dict(color='Green',dash="dot"),
                xref='x',
                yref='y',
                row=1,
                col=1,
    )
    
    fig.add_trace(go.Scatter(mode="markers+lines", x=epochs, y=hist["loss"], name = "Loss"), row=1, col=2)
    fig.add_trace(go.Scatter(mode="markers+lines", x=epochs, y=hist["val_loss"], name = "Val loss"), row=1, col=2)
    
    fig.add_shape(type='line',
                x0=0,
                y0=np.min(hist["val_loss"]),
                x1=np.max(epochs),
                y1=np.min(hist["val_loss"]),
                line=dict(color='Green',dash="dot"),
                xref='x',
                yref='y',
                row=1,
                col=2,
    )
    
    fig['layout']['xaxis']['title']='Epoch'
    fig['layout']['xaxis2']['title']='Epoch'
    fig['layout']['yaxis']['title']='Accuracy'
    fig['layout']['yaxis2']['title']='Loss'
    
    return fig

In [ ]:
def tensor_mod(a, n):
    a = tf.cast(a, dtype=tf.float32)
    n = tf.cast(n, dtype=tf.float32)
    
    return a - tf.multiply(n, tf.math.floor(a/n))

def delta_phi(a, b):
    #a = np.asarray(a)
    #b = np.asarray(b)
    #x = (a - b + np.pi) % (2 * np.pi) - np.pi
    pi  = tf.constant([np.pi])
    x = tf.subtract(a, b)
    x += np.pi
    #x = x % (2 * np.pi)
    n = tf.constant(2*np.pi)
    x = tensor_mod(x, n)
    
    x -=np.pi
    return x


def delta_r2(eta1, phi1, eta2, phi2):
    return tf.square(eta1 - eta2) + tf.square(delta_phi(phi1, phi2))

def to_pt2(x, eps=1e-8):
    pt2 = tf.reduce_sum(tf.square(x[:, :2]), axis=1, keepdims=True)
    if eps is not None:
        pt2 = tf.clip_by_value(pt2, clip_value_min=eps, clip_value_max=10e32)
    return pt2

def atan2(y, x):
    sx = tf.math.sign(x)
    sy = tf.math.sign(y)
    pi_part = (sy + sx * (sy ** 2 - 1)) * (sx - 1) * (-math.pi / 2)
    atan_part = tf.math.atan(y / (x + (1 - sx ** 2))) * sx ** 2
    return atan_part + pi_part


def to_ptrapphim(x, return_mass=True, eps=1e-8, for_onnx=False):
    # x: (N, 4, ...), dim1 : (px, py, pz, E)
    
    
    px, py, pz, energy = tf.split(x, (1, 1, 1, 1), axis=1)
    pt = tf.sqrt(to_pt2(x, eps=eps))
    # rapidity = 0.5 * torch.log((energy + pz) / (energy - pz))
    rapidity = 0.5 * tf.math.log(1 + (2 * pz) / (energy - pz))
    sign = tf.math.sign(rapidity)
    rapidity =  tf.clip_by_value(tf.abs(rapidity), clip_value_min=1e-20, clip_value_max=10e32)
    rapidity = sign*rapidity
    
    phi = (atan2 if for_onnx else tf.math.atan2)(py, px)
    if not return_mass:
        return tf.concat((pt, rapidity, phi), axis=1)
    else:
        m = tf.sqrt(to_m2(x, eps=eps))
        return tf.concat((pt, rapidity, phi, m), axis=1)


def boost(x, boostp4, eps=1e-8):
    # boost x to the rest frame of boostp4
    # x: (N, 4, ...), dim1 : (px, py, pz, E)
    p3 = tf.clip_by_value(-boostp4[:, :3] / boostp4[:, 3:], clip_value_min=eps, clip_value_max=10e32)
    #p3 = -boostp4[:, :3] / boostp4[:, 3:].clamp(min=eps)
    b2 = tf.reduce_sum(tf.square(p3), axis = 1, keepdims=True)
    #b2 = p3.square().sum(dim=1, keepdim=True)
    gamma = tf.sqrt(tf.clip_by_value(1 - b2, clip_value_min=eps, clip_value_max=10e32))
    #gamma = (1 - b2).clamp(min=eps)**(-0.5)
    gamma2 = (gamma - 1) / b2
    gamma2 = tf.where(b2==0, 0, gamma2)
    #gamma2.masked_fill_(b2 == 0, 0)
    bp = tf.reduce_sum(x[:, :3] * p3, axis = 1, keepdims=True)
    #bp = (x[:, :3] * p3).sum(dim=1, keepdim=True)
    v = x[:, :3] + gamma2 * bp * p3 + x[:, 3:] * gamma * p3
    return v


def p3_norm(p, eps=1e-8):
    #p[:, :3] / p[:, :3].norm(dim=1, keepdim=True).clamp(min=eps)
    return tf.clip_by_value(p[:, :3] / tf.norm(p[:, :3], axis=1, keepdims=True), clip_value_min=eps, clip_value_max=10e32)

def to_m2(x, eps=1e-8):
    m2 = tf.square(x[:, :, 3:4]) - tf.reduce_sum(tf.square(x[:, :, :3]), axis=-1, keepdims=True)
    #m2 = x[:, 3:4].square() - x[:, :3].square().sum(dim=1, keepdim=True)
    if eps is not None:
        m2 = tf.clip_by_value(m2, clip_value_min=eps, clip_value_max=10e32)
    return m2

In [ ]:
def to_pt2_pre(x, eps=1e-8):
    pt2 = tf.reduce_sum(tf.square(x[:, :, :2]), axis=2, keepdims=True)
    if eps is not None:
        pt2 = tf.clip_by_value(pt2, clip_value_min=eps, clip_value_max=10e32)
    return pt2

def to_ptrapphim_pre(x, return_mass=True, eps=1e-8, for_onnx=False):
    # x: (N, 4, ...), dim1 : (px, py, pz, E)
    #print(x)
    #x = tf.reshape(x, (tf.shape(x)[2],tf.shape(x)[1],tf.shape(x)[0]))
    #print(x)
    px, py, pz, energy = tf.split(x, (1, 1, 1, 1), axis=2)
    pt = tf.sqrt(to_pt2_pre(x, eps=eps))
    # rapidity = 0.5 * torch.log((energy + pz) / (energy - pz))
    rapidity = 0.5 * tf.math.log(1 + (2 * pz) / (energy - pz))
    sign = tf.math.sign(rapidity)
    rapidity =  tf.clip_by_value(tf.abs(rapidity), clip_value_min=1e-20, clip_value_max=10e32)
    rapidity = sign*rapidity
    
    phi = (atan2 if for_onnx else tf.math.atan2)(py, px)
    if not return_mass:
        return pt, rapidity, phi
    else:
        m = tf.sqrt(to_m2(x, eps=eps))
        return tf.concat((pt, rapidity, phi, m), axis=1)

def import_top_data(filename, dataset_size, alpha = 0., eps=1e-8, padding=200):
    print("Importing data...\n")
    path = "TopTagging/{}.h5".format(filename)
    df = pd.read_hdf(path, 'table')
    
    part_px = []
    part_py = []
    part_pz = []
    part_E = []
    
#     jet_px=df["truthPX"][:dataset_size].to_list()
#     jet_py=df["truthPY"][:dataset_size].to_list()
#     jet_pz=df["truthPZ"][:dataset_size].to_list()
#     jet_E=df["truthE"][:dataset_size].to_list()

    if padding > 200: padding=200
    
    for i in range(padding):
        part_px.append(df["PX_{}".format(i)][:dataset_size].to_list())
        part_py.append(df["PY_{}".format(i)][:dataset_size].to_list())
        part_pz.append(df["PZ_{}".format(i)][:dataset_size].to_list())
        part_E.append(df["E_{}".format(i)][:dataset_size].to_list())


    rot_mat = [[1.,0.,0.,0.],
           [0.,1.,0.,0.],
           [0.,0.,np.cos(alpha), -np.sin(alpha)],
           [0.,0.,np.sin(alpha), np.cos(alpha)]]
    rot_tensor = tf.constant(rot_mat)
    
    
    part_px = np.stack(part_px, axis=-1)
    part_py = np.stack(part_py, axis=-1)
    part_pz = np.stack(part_pz, axis=-1)
    part_E = np.stack(part_E, axis=-1)   
     
    if alpha != 0:
        part_py_p = np.cos(alpha)*part_py - np.sin(alpha)*part_pz
        part_pz_p = np.cos(alpha)*part_pz + np.sin(alpha)*part_py
        part_py = part_py_p
        part_pz = part_pz_p
    
    jet_px = np.sum(np.asarray(part_px),axis=1)
    jet_py= np.sum(np.asarray(part_py),axis=1)
    jet_pz= np.sum(np.asarray(part_pz),axis=1)
    jet_E= np.sum(np.asarray(part_E),axis=1)        
        
    batch_part = tf.stack([part_px, part_py, part_pz, part_E], axis=-1)
    batch_jet = tf.stack([jet_px, jet_py, jet_pz, jet_E], axis=-1)
    
    batch_jet = tf.reshape(batch_jet, (tf.shape(batch_jet)[0],1,tf.shape(batch_jet)[1]))
    batch_jet = tf.tile(batch_jet, (1,padding, 1))

    part_pt, part_rapidity, part_phi = to_ptrapphim_pre(batch_part, return_mass=False, eps=1e-8, for_onnx=False)
    jet_pt, jet_rapidity, jet_phi= to_ptrapphim_pre(batch_jet, return_mass=False, eps=1e-8, for_onnx=False)
    
    jet_E = tf.reshape(jet_E, (tf.shape(jet_E)[0],1))
    jet_E = tf.tile(jet_E, (1,padding))

    part_eta = part_rapidity - jet_rapidity
    part_phi = delta_phi(part_phi, jet_phi)
    part_logpt = tf.clip_by_value(tf.math.log(part_pt), clip_value_min=eps, clip_value_max=10e32)
    part_logE = tf.math.sign(tf.math.log(part_E))*(tf.clip_by_value(tf.abs(tf.math.log(part_E)), clip_value_min=-eps, clip_value_max=10e32))
    part_ptptjet =  part_pt/jet_pt 
    part_EEjet =  part_E/jet_E 
    part_dR = tf.sqrt(tf.square(tf.cast(part_eta,dtype=tf.float32)) + tf.square(tf.cast(part_phi,dtype=tf.float32)))
    y = df['is_signal_new'][:dataset_size].to_list()
    
    batch = np.stack([part_px, part_py, part_pz, part_E, tf.squeeze(part_eta), tf.squeeze(part_phi), tf.squeeze(part_logpt),
                           tf.squeeze(part_logE), tf.squeeze(part_ptptjet), tf.squeeze(part_EEjet), tf.squeeze(part_dR)], axis=-1)
    batch = tf.convert_to_tensor(batch)
    
    padding_bool = tf.reduce_all(batch[:,:,:4]!=[0.,0.,0.,0.],axis=-1) 
    padding_arr = tf.cast(padding_bool, dtype=tf.float64)

    batch = tf.einsum('...i,...ij->...ij', padding_arr, batch)

    batch = tf.where(tf.math.is_nan(batch), tf.zeros_like(batch), batch)

    batch = tf.cast(batch,dtype=tf.float32)
    print("Data imported.\n")
    return batch, tf.convert_to_tensor(y,dtype=tf.float32)

In [ ]:
num_partons = 128
training_data = import_top_data('train', dataset_size=100000,padding=num_partons)

In [ ]:
def pairwise_lv_fts_old(input_data, output_vars=['kt', 'z', 'delta', 'm2'], eps=1e-8, for_onnx=False):

    
    x = input_data[:,:,:4]
    y = tf.TensorArray(dtype=tf.float32, size=1, dynamic_size=True)
    batch_size = tf.squeeze(tf.shape(x))[0]
    length = tf.squeeze(tf.shape(x))[1]
    
    num_outputs = len(output_vars)
    
    padding_bool = tf.reduce_all(x!=[0,0,0,0], axis=-1) 
    padding_arr = tf.cast(padding_bool, dtype=tf.float32)
    padding_mat = tf.einsum('...i,...j->...ij', padding_arr, padding_arr)
    outputs = tf.TensorArray(dtype=tf.float32, size=num_outputs)
    
    for i in range(batch_size):
        all_out = []
        z = x[i]
    


        xi = tf.expand_dims(z,0)
        xj = tf.expand_dims(z,1)
        pt, rap, phi = tf.split(to_ptrapphim(z, False, eps=None, for_onnx=for_onnx), (1, 1, 1), axis=1)
        pti = tf.expand_dims(pt,0)
        rapi = tf.expand_dims(rap,0)
        phii = tf.expand_dims(phi,0)
        
        ptj = tf.expand_dims(pt,1)
        rapj = tf.expand_dims(rap,1)
        phij = tf.expand_dims(phi,1)
        delta = tf.sqrt(delta_r2(rapi, phii, rapj, phij))
        lndelta = tf.math.log(tf.clip_by_value(delta, clip_value_min=eps, clip_value_max=10e32))
        
        ptmin = ((pti <= ptj) * pti + (pti > ptj) * ptj) if for_onnx else tf.math.minimum(pti, ptj)
        lnkt = tf.clip_by_value(tf.math.log(ptmin * delta), clip_value_min=eps, clip_value_max=10e32)

        ptmin_sum = tf.clip_by_value(ptmin / (pti + ptj), clip_value_min=eps, clip_value_max=10e32)
        lnz = tf.clip_by_value(tf.math.log(ptmin_sum), clip_value_min=eps, clip_value_max=10e32)        
        xij = xi + xj
        lnm2 = tf.math.log(to_m2(xij, eps=eps))
        

        
        kinematic_dict =  dict([('kt', lnkt) ,('z', lnz) ,('delta', lndelta) , ('m2', lnm2)])
        
        for i in range(num_outputs):
            temp = tf.transpose(kinematic_dict[output_vars[i]])
            outputs.write(i, temp)
    
        y.write(i, tf.transpose(outputs.stack()))
    
    
    y = y.stack()
    y = tf.reshape(y, shape=(tf.shape(y)[0], tf.shape(y)[1], tf.shape(y)[2], tf.shape(y)[4]))
    out = tf.einsum('...ij,...ijk->...ijk', padding_mat, y)
    out = tf.where(tf.math.is_nan(out), tf.zeros_like(out), out)
    
    return out

In [ ]:
def to_m2_test(x, eps=1e-8):
    m2 = tf.square(x[:, :, :, 3:4]) - tf.reduce_sum(tf.square(x[:, :, :, :3]), axis=-1, keepdims=True)
    #m2 = x[:, 3:4].square() - x[:, :3].square().sum(dim=1, keepdim=True)
    if eps is not None:
        m2 = tf.clip_by_value(m2, clip_value_min=eps, clip_value_max=10e32)
    return m2

def pairwise_lv_fts(input_data, output_vars=['kt', 'z', 'delta', 'm2'], eps=1e-8, for_onnx=False):

    
    x = input_data[:,:,:4]
    batch_size = tf.squeeze(tf.shape(x))[0]
    length = tf.squeeze(tf.shape(x))[1]
    
    num_outputs = len(output_vars)
    
    padding_bool = tf.reduce_all(x!=[0,0,0,0], axis=-1) 
    padding_arr = tf.cast(padding_bool, dtype=tf.float32)
    padding_mat = tf.einsum('...i,...j->...ij', padding_arr, padding_arr)
    outputs = []
    
    all_out = []

    xi = tf.expand_dims(x,1)
    xj = tf.expand_dims(x,2)
    pt_rap  = to_ptrapphim_pre(x, False, eps=None, for_onnx=for_onnx)
    pt, rap, phi = tf.split(to_ptrapphim_pre(x, False, eps=None, for_onnx=for_onnx), (1, 1, 1), axis=0)
    
    pt = tf.squeeze(pt)
    rap = tf.squeeze(rap)
    phi = tf.squeeze(phi)
    

    
    pti = tf.expand_dims(pt,1)
    rapi = tf.expand_dims(rap,1)
    phii = tf.expand_dims(phi,1)
    pti = tf.expand_dims(pti,3)
    rapi = tf.expand_dims(rapi,3)
    phii = tf.expand_dims(phii,3)
    
    ptj = tf.expand_dims(pt,2)
    rapj = tf.expand_dims(rap,2)
    phij = tf.expand_dims(phi,2)
    ptj = tf.expand_dims(ptj,3)
    rapj = tf.expand_dims(rapj,3)
    phij = tf.expand_dims(phij,3)
    
    
    delta = tf.sqrt(delta_r2(rapi, phii, rapj, phij))
    lndelta = tf.math.log(tf.clip_by_value(delta, clip_value_min=eps, clip_value_max=10e32))

    ptmin = ((pti <= ptj) * pti + (pti > ptj) * ptj) if for_onnx else tf.math.minimum(pti, ptj)
    lnkt = tf.clip_by_value(tf.math.log(ptmin * delta), clip_value_min=eps, clip_value_max=10e32)

    ptmin_sum = tf.clip_by_value(ptmin / (pti + ptj), clip_value_min=eps, clip_value_max=10e32)
    lnz = tf.clip_by_value(tf.math.log(ptmin_sum), clip_value_min=eps, clip_value_max=10e32)        
    xij = xi + xj
    lnm2 = tf.math.log(to_m2_test(xij, eps=eps))

    kinematic_dict =  dict([('kt', lnkt) ,('z', lnz) ,('delta', lndelta) , ('m2', lnm2)])

    for i in range(num_outputs):
        outputs.append(kinematic_dict[output_vars[i]])

    y = tf.stack(outputs)
    y = tf.squeeze(y)
    
    if num_outputs == 1:
        y = tf.expand_dims(y, 0)

    y = tf.transpose(y, perm=(1,2,3,0))
    out = tf.einsum('...ij,...ijk->...ijk', padding_mat, y)
    out = tf.where(tf.math.is_nan(out), tf.zeros_like(out), out)
    return out

In [ ]:
class Embedding(tf.keras.layers.Layer):
    def __init__(self, max_len, d_model, input_dim, activation = 'gelu'):
        super().__init__()
        self.d_model = d_model
        self.input_dim = input_dim
        self.max_len = max_len
        self.masking = tf.keras.layers.Masking()
        self.batchnorm = tf.keras.layers.BatchNormalization()
        layer_arr = []
        for dim in d_model:
            temp = [tf.keras.layers.LayerNormalization(), tf.keras.layers.Dense(dim,activation=activation)]
            layer_arr.append(temp)
        
        self.layers = layer_arr
        
    def call(self, x):
        #x = self.dense(x)
        x = self.masking(x)
        x = self.batchnorm(x)
        for layer in self.layers:
            x = layer[0](x)
            x = layer[1](x)
        return x
    
class PairEmbed(tf.keras.layers.Layer):
    def __init__(self, max_len, output_vars, d_model, activation = 'gelu', eps=1e-8, for_onnx=False):
        super().__init__()
        self.for_onnx = for_onnx
        self.pairwise_lv_fts = partial(pairwise_lv_fts, output_vars = output_vars, eps=eps, for_onnx = for_onnx)
        self.masking = tf.keras.layers.Masking()
        self.num_outputs = len(output_vars)
        self.batchnorm = tf.keras.layers.BatchNormalization()
        
        layers_arr = []
        for dim in d_model:
            temp = [tf.keras.layers.Conv1D(filters=dim,kernel_size=1),tf.keras.layers.BatchNormalization(), tf.keras.layers.Activation('gelu')]
            layers_arr.append(temp)
            
        self.layers = layers_arr
        self.output_dim = d_model[-1]

    def call(self, x):
        # x: (batch, v_dim, seq_len)
        batch_size = tf.shape(x)[0]
        n_particles = tf.shape(x)[1]
        seq_len = tf.shape(x)[2]
        x = self.masking(x)
        x = self.pairwise_lv_fts(x)
        x = tf.reshape(x, (batch_size, n_particles * n_particles, 1, self.num_outputs))
        x = self.batchnorm(x)
        
        for layer in self.layers:
            x = layer[0](x)
            x = layer[1](x)
            x = layer[2](x)
        
        x = tf.reshape(x, (batch_size, n_particles, n_particles, self.output_dim))
        x = self.masking(x)
        return x
    
class ScalingLayer(tf.keras.layers.Layer):
    def __init__(self,):
        super().__init__()
        self.initial_value = 1.
        self.learned = tf.Variable(self.initial_value, name = 'learned_scalar')
    def call(self, x):
        return self.learned*x
    
class DotProductAttention(tf.keras.layers.Layer):
    def __init__(self,dropout=0.0, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        self.masking = tf.keras.layers.Masking()
        self.softmax = tf.keras.layers.Softmax()
        self.dropout = tf.keras.layers.Dropout(dropout)
    
    def compute_mask(self, inputs, mask=None):
        return mask
    
    def call(self, queries, keys, values, d_k, U,  mask=None):
        
        U = tf.transpose(U, perm = (0,3,1,2))
        U = self.masking(U)
        
        scores = tf.matmul(queries, keys, transpose_b=True)/tf.math.sqrt(tf.cast(d_k, tf.float32))
        scores = self.masking(scores)

        scores = tf.add(scores, U)
        scores = self.masking(scores)
        
        masked_inp = tf.where(scores!=0, scores, tf.float64.min)
        weights = self.softmax(masked_inp)
        
        index_nan = tf.math.is_nan(weights)
        shape = U.shape
        
        #zeroes = tf.zeros(shape = (shape[0],shape[1], shape[2], shape[3]))
        #weights = weights.numpy()
        #weights[np.isnan(weights)] = 0
        
        
        #weights = tf.tensor_scatter_nd_update(weights, tf.expand_dims(index_nan, 1), tf.gather(zeroes, index_nan))
        #weights = tf.constant(weights)
        weights= self.masking(weights)
        weights = self.dropout(weights)
        
        values = self.masking(values)
        
        outputs = tf.linalg.matmul(weights, values)
        outputs = self.masking(outputs)
        return outputs
        

class PMultiHeadedAttention(tf.keras.layers.Layer):
    ### Adapted from machinelearningmastery.com/how-to-implement-multi-head-attention-from-scratch-in-tensorflow-and-keras/
    
    def __init__(self, heads, d_k, d_v, d_model, dropout=0.0, **kwargs):
        super(PMultiHeadedAttention, self).__init__(**kwargs)
        self.attention = DotProductAttention(dropout=dropout)  # Scaled dot product attention
        self.heads = heads  # Number of attention heads to use
        self.d_k = d_k  # Dimensionality of the linearly projected queries and keys
        self.d_v = d_v  # Dimensionality of the linearly projected values
        self.d_model = d_model  # Dimensionality of the model
        self.masking = tf.keras.layers.Masking()
        self.W_q = tf.keras.layers.Dense(d_k)  # Learned projection matrix for the queries
        self.W_k = tf.keras.layers.Dense(d_k)  # Learned projection matrix for the keys
        self.W_v = tf.keras.layers.Dense(d_k)  # Learned projection matrix for the values
        self.W_o = tf.keras.layers.Dense(d_model)
        
    def compute_mask(self, inputs, mask=None):
        return mask
    
    def reshape_tensor(self, x, heads,  d_model, flag):
        if flag:
            
            x = tf.reshape(x, shape=(tf.shape(x)[0],tf.shape(x)[1], heads, -1))
            x = tf.transpose(x, perm = (0,2,1,3))

        else:
            x = tf.transpose(x, perm = (0,2,1,3))
            x = tf.reshape(x, shape=(tf.shape(x)[0],tf.shape(x)[1], d_model))
        
        return x
    
    def call(self, x, keys, values, U, mask=None):
        # Rearrange the queries to be able to compute all heads in parallel
        # Compute the multi-head attention output using the reshaped queries, keys and values
        d_model = tf.shape(x)[-1]
        x = self.attention(self.reshape_tensor(self.W_q(x), self.heads, d_model, True), self.reshape_tensor(self.W_k(keys), self.heads, d_model, True), self.reshape_tensor(self.W_v(values), self.heads, d_model, True), self.d_k, U, mask)
        # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)
        x = self.masking(x)
        # Rearrange back the output into concatenated form
        x = self.reshape_tensor(x, self.heads, d_model, False)
        # Resulting tensor shape: (batch_size, input_seq_length, d_v)
        
        x = self.masking(x)
        # Apply one final linear projection to the output to generate the multi-head attention
        # Resulting tensor shape: (batch_size, input_seq_length, d_model)
        x = self.W_o(x)

        return x
    
class BaseAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, key_dim, num_heads, dropout, **kwargs):
        super().__init__()
        self.embed_dim = embed_dim
        self.add = tf.keras.layers.Add()
        self.gelu = tf.keras.layers.Activation(tf.keras.activations.gelu)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.linear = tf.keras.layers.Dense(embed_dim, activation=None)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim = key_dim, dropout=dropout)
        self.p_mha = KerasPMHA(num_heads=num_heads, key_dim = key_dim, dropout=dropout)# self.p_mha = PMultiHeadedAttention(num_heads, key_dim, key_dim, embed_dim, dropout=dropout, **kwargs) 
        self.dropout = tf.keras.layers.Dropout(dropout)
        
class PAttentionBlock(BaseAttention):
    def call(self, x, context):
        x_context = x
        x = self.layernorm(x)
        x = self.p_mha(x,x,x,bias_mask=context)# x = self.p_mha(x=x, keys=x, values=x, U=context)
        x = self.layernorm(x)
        x = self.dropout(x)
        x = self.add([x, x_context])
        
        x_context = x
        x = self.layernorm(x)
        x = self.linear(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.layernorm(x)
        x = self.linear(x)
        x = self.dropout(x)
        x = self.add([x, x_context])
        return x
    
class PlainAttentionBlock(BaseAttention):
    def call(self, x):
        x_context = x
        x = self.layernorm(x)
        x = self.mha(x,x,x)
        x = self.layernorm(x)
        x = self.dropout(x)
        x = self.add([x, x_context])
        
        x_context = x
        x = self.layernorm(x)
        x = self.linear(x)
        x = self.gelu(x)
        x = self.dropout(x)
        x = self.layernorm(x)
        x = self.linear(x)
        x = self.dropout(x)
        x = self.add([x, x_context])
        return x

class CAttentionBlock(BaseAttention):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def call(self,x, x_cls):
        x = tf.concat((x_cls, x), axis=1)
        x = self.layernorm(x)
        x = self.mha(x_cls, x, x)
        x = self.layernorm(x)
        x = self.add([x, x_cls])
        
        x_context = x
        x = self.layernorm(x)
        x = self.linear(x)
        x = self.gelu(x)
        x = self.layernorm(x)
        x = self.linear(x)
        x = self.add([x, x_context])
        return x
    
class MLP(tf.keras.layers.Layer):
    def __init__(self, MLP_params):
        super().__init__()
        layer_arr = [] 
        for out_dim, drop_rate in MLP_params:
            temp = [tf.keras.layers.Dense(out_dim, activation=None),tf.keras.layers.Dropout(rate = drop_rate)]
            layer_arr.append(temp)
            
        self.MLP_layers = layer_arr
    def call(self, x):
        for layers in self.MLP_layers:
            x = layers[0](x)
            x = layers[1](x)
        return x

class ParticleTransformer(tf.keras.Model):
    def __init__(self, d_model, d_pair_model, max_len, input_dim, output_vars, num_heads, N_p, N_c, output_dims, isPlain = False, **kwargs):
        super().__init__(**kwargs)
        d_pair_model.append(num_heads)
        self.embed_dim = d_model[-1]
        self.embed = Embedding(max_len, d_model, input_dim)
        self.pair_embed = PairEmbed(max_len, output_vars, d_pair_model)
        self.isPlain = isPlain
        
        pattention = []
        if isPlain:
            for i in range(N_p):
                pattention.append(PlainAttentionBlock(embed_dim = self.embed_dim,key_dim=self.embed_dim, num_heads=num_heads, dropout=0.1))
            self.particle_attention_arr = pattention
        
        else:
            for i in range(N_p):
                pattention.append(PAttentionBlock(embed_dim = self.embed_dim,key_dim=self.embed_dim, num_heads=num_heads, dropout=0.1))
            self.particle_attention_arr = pattention
        
        cattention = []
        for i in range(N_c):
            cattention.append(CAttentionBlock(embed_dim = self.embed_dim,key_dim=self.embed_dim, num_heads=num_heads, dropout=0.0))
        self.class_attention_arr = cattention

        self.cls_tkn = tf.Variable(name="class token", trainable=True, initial_value = tf.random.normal(shape=(1,1,self.embed_dim),stddev=0.2))
        self.mlp = MLP(output_dims)
        self.softmax = tf.keras.layers.Softmax()
        self.sigmoid = tf.keras.layers.Activation(tf.keras.activations.sigmoid)
        self.n_p = N_p
        self.n_c = N_c
        self.output_dims = output_dims
        
    def call(self, inputs):
        x = self.embed(inputs)
        if self.isPlain:
            for block in self.particle_attention_arr:
                x = block(x)
        
        else:
            U = self.pair_embed(inputs)

            for block in self.particle_attention_arr:
                x = block(x, U)
        
        broadcast_shape = (tf.shape(x)[0], 1, 1)
        class_tkn = self.cls_tkn
        class_tkn = tf.tile(class_tkn, broadcast_shape)
        #class_tkn = tf.broadcast_to(self.cls_tkn, broadcast_shape)
        x_context = x
        x = class_tkn

        for block in self.class_attention_arr:
            x = block(x_context, x)

        x = tf.reshape(x, (tf.shape(x)[0], tf.shape(x)[2]))
        #x = tf.squeeze(x)    
        x = self.mlp(x)
        
        if self.output_dims[-1][0] == 1:
            x = self.sigmoid(x)
        else:
            x = self.softmax(x)
        
        return x

In [ ]:
class ParticleTransformerWrapper():
    def __init__(self, d_model, d_pair_model, max_number_partons, input_dim, pairwise_outputs, num_heads, N_p, N_c, output_dims, isPlain = False, train_filename = 'train', model_name = "ParticleTransformer",  dataset_size = 10000, **kwargs):
        self.ParT = ParticleTransformer(d_model, d_pair_model, max_number_partons, input_dim, pairwise_outputs,  num_heads, N_p, N_c, output_dims, isPlain=isPlain, **kwargs)
        self.input_dim = input_dim
        self.padding = max_number_partons
        self.model_name = model_name
        self.weight_filename = model_name
        self.weight_log = []
        self.val_acc_loss_log = []
        self.is_built = False
        self.out_dim = output_dims[-1][0]
        
    def build(self):
        input_shape = (None, self.padding, self.input_dim)
        RAdam = tfa.optimizers.RectifiedAdam(beta_1=0.95, beta_2=0.999, epsilon=0.00001, weight_decay=0.01)
        Lookahead = tfa.optimizers.Lookahead(RAdam)
        
        if self.out_dim == 1:
            loss_func = 'binary_crossentropy'
        else: loss_func = 'categorical_crossentropy'
        print(loss_func)
        
        self.ParT.compile(
             loss=loss_func,
             optimizer=Lookahead,
             metrics=['Accuracy'],
            )
        self.ParT.build(input_shape)
        self.is_built = True
    
    def summary(self):
        if self.is_built:
            return self.ParT.summary()
        else:
            print("This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.")
            return None
    
    def train(self, dataset, dynamic_lr = False, initial_lr=0.001, max_lr = 0.01, min_max_ratio = 0.1, rampup_epochs = 0,sustain_ratio = 0.7, early_stopping = True, print_summary = False, train_val_split = 0.3, metrics = ['Accuracy'], epochs = 20, batch_size = 8, learning_rate = 0.001):
        print("Loading data...\n") 
        X, y = dataset
        X_train, X_test, y_train, y_test = train_test_split(X.numpy(), y.numpy(), shuffle=True, test_size=train_val_split)
        X_train = tf.convert_to_tensor(X_train)
        X_test = tf.convert_to_tensor(X_test)
        y_train = tf.convert_to_tensor(y_train)
        y_test = tf.convert_to_tensor(y_test)
        print("Loaded data.\nCompiling model...\n")
        
        RAdam = tfa.optimizers.RectifiedAdam(beta_1=0.95, beta_2=0.999, epsilon=0.00001)#, weight_decay=0.01)
        Lookahead = tfa.optimizers.Lookahead(RAdam)
        
        
        if self.out_dim == 1:
            loss_func = 'binary_crossentropy'
        else: loss_func = 'categorical_crossentropy'
        
        self.ParT.compile(
             loss=loss_func,
             optimizer=Lookahead,
             metrics=metrics,
            )

        print("Model compiled.\nBeginning training... \n")
        
        path = "Checkpoints/{}".format(self.model_name)
        path = path+"/cp-{epoch:04d}"

              
        min_lr = min_max_ratio*max_lr

        sustain_epochs = int(sustain_ratio*epochs)
        exp_decay = np.exp(-1)
        if rampup_epochs != 0:
            grad = (max_lr - initial_lr)/rampup_epochs
        else: grad=0
        
        def lrfn(epoch):
            
            if epoch < rampup_epochs:
                return initial_lr + grad*epoch
            elif epoch < sustain_epochs+rampup_epochs:
                return max_lr
            else:
                return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr

        lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True) 

        self.ParT.save_weights(path.format(epoch=0))
        cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=path,
        verbose=1,
        save_weights_only=True,
        monitor='val_Accuracy',
        mode='max',
        save_best_only=True)
        
        es_loss = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=5)
        log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    
        if early_stopping and dynamic_lr:
            
            history = self.ParT.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[es_loss, lr_callback, cp_callback,tensorboard_callback], validation_data=(X_test, y_test), shuffle=True)

        if early_stopping:
            history = self.ParT.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[es_loss, cp_callback,tensorboard_callback], validation_data=(X_test, y_test), shuffle=True)

        if dynamic_lr:
            history = self.ParT.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[lr_callback, cp_callback,tensorboard_callback], validation_data=(X_test, y_test), shuffle=True)
 
        else:
            history = self.ParT.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[cp_callback,tensorboard_callback], validation_data=(X_test, y_test), shuffle=True)



        self.save_weights()
        self.is_built = True
        
        if print_summary == True:
            print(self.ParT.summary())
        
        figure = histplot(history)
        figure.show()
        filename = self.model_name
        isFile = os.path.isfile("Figures/{}.png".format(filename))
        i=1
        while isFile:
            filename = "{}{}".format(self.model_name, i)
            i+=1
            isFile = os.path.isfile("Figures/{}.png".format(filename))

        figure.write_image("Figures/{}.png".format(filename), width=800, height=500)
        
        return
    
    def test(self, dataset, get_score=True, percent_of_total_shown=0.1, batch_size=8):
        X, y = dataset
        y_pred = self.ParT.predict(X, batch_size=batch_size)
        if get_score:
            try:
                score = self.ParT.evaluate(X, y, batch_size=batch_size, verbose=1)
                print('Test loss:', score[0])
                print('Test accuracy:', score[1])
            except(RuntimeError):
                print("Cannot evaluate model without first compiling the model for training/testing")

        fig = make_subplots(rows=1, cols=1,subplot_titles=(['ROC']))
        try:
            fpr, tpr, thresholds = metrics.roc_curve(y, y_pred)
            AUC = metrics.auc(fpr, tpr)
            reduced_fpr = []
            reduced_tpr = []
            reduced_threshold = []
            for i in range(len(fpr)):
                if np.random.rand()<percent_of_total_shown and i<len(thresholds):
                    reduced_fpr.append(fpr[i])
                    reduced_tpr.append(tpr[i])
                    reduced_threshold.append(thresholds[i])


            fig.add_trace(go.Scatter(x=reduced_fpr, y=reduced_tpr,
                                    mode='lines',
                                    marker_line_color="midnightblue", 
                                    text = reduced_threshold,
                                    hovertemplate = 'Thresh: %{text:.3f}<extra></extra>',
                                    ), row=1, col=1)
            
        except(ValueError):
            print("Could not compute ROC due NaN values in y_pred")
            AUC = np.NaN
        fig['layout']['xaxis1']['title']='FPR'
        fig['layout']['yaxis1']['title']='TPR'
        
        return fig, AUC
    
    def save_weights(self):
        i=1
        filename = self.model_name
        isFile = os.path.isfile("Weights/{}.index".format(filename))
        while isFile:
            filename = "{}{}".format(self.model_name, i)
            isFile = os.path.isfile("Weights/{}.index".format(filename))
            i+=1
        
        self.ParT.save_weights("Weights/{}".format(filename))
        self.weight_filename = filename
    
    def load_weights(self, filepath):
        isFile = os.path.isfile(filepath+".index")
        if isFile:
            try:
                self.ParT.load_weights(filepath)
            except(...):
                print("Error loading weights. Check 'filename'.data-00000-of-00001 exists in the weights folder.")
        else: print("{} does not appear to exist.".format(filepath))          

In [ ]:
ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full2')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain2')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m22')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta2')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt2')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z2')

NameError: ignored

In [ ]:
batch_size = 256
sustain_ratio = 0.7

ParT_Full.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.3733 - Accuracy: 0.8544
Epoch 1: val_Accuracy improved from -inf to 0.51540, saving model to Checkpoints/ParT_Full2/cp-0001
274/274 [==============================] - 203s 511ms/step - loss: 0.3733 - Accuracy: 0.8544 - val_loss: 1.2290 - val_Accuracy: 0.5154 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.2410 - Accuracy: 0.9035
Epoch 2: val_Accuracy improved from 0.51540 to 0.85717, saving model to Checkpoints/ParT_Full2/cp-0002
274/274 [==============================] - 135s 494ms/step - loss: 0.2410 - Accuracy: 0.9035 - val_loss: 0.3176 - val_Accuracy: 0.8572 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [=======

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.4729 - Accuracy: 0.7977
Epoch 1: val_Accuracy improved from -inf to 0.83740, saving model to Checkpoints/ParT_Plain2/cp-0001
274/274 [==============================] - 122s 277ms/step - loss: 0.4729 - Accuracy: 0.7977 - val_loss: 0.3681 - val_Accuracy: 0.8374 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3067 - Accuracy: 0.8731
Epoch 2: val_Accuracy did not improve from 0.83740
274/274 [==============================] - 72s 263ms/step - loss: 0.3067 - Accuracy: 0.8731 - val_loss: 0.4408 - val_Accuracy: 0.8275 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [==============================] - ETA: 0s - loss: 0.2719 - 

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.3733 - Accuracy: 0.8529
Epoch 1: val_Accuracy improved from -inf to 0.51553, saving model to Checkpoints/ParT_m22/cp-0001
274/274 [==============================] - 192s 514ms/step - loss: 0.3733 - Accuracy: 0.8529 - val_loss: 1.8355 - val_Accuracy: 0.5155 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.2522 - Accuracy: 0.8993
Epoch 2: val_Accuracy did not improve from 0.51553
274/274 [==============================] - 137s 499ms/step - loss: 0.2522 - Accuracy: 0.8993 - val_loss: 2.6181 - val_Accuracy: 0.4950 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [==============================] - ETA: 0s - loss: 0.2157 - Ac

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.4808 - Accuracy: 0.8023
Epoch 1: val_Accuracy improved from -inf to 0.70147, saving model to Checkpoints/ParT_delta2/cp-0001
274/274 [==============================] - 190s 503ms/step - loss: 0.4808 - Accuracy: 0.8023 - val_loss: 0.6697 - val_Accuracy: 0.7015 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.2835 - Accuracy: 0.8856
Epoch 2: val_Accuracy improved from 0.70147 to 0.88057, saving model to Checkpoints/ParT_delta2/cp-0002
274/274 [==============================] - 134s 490ms/step - loss: 0.2835 - Accuracy: 0.8856 - val_loss: 0.2956 - val_Accuracy: 0.8806 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [=====

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.3763 - Accuracy: 0.8496
Epoch 1: val_Accuracy improved from -inf to 0.49847, saving model to Checkpoints/ParT_kt2/cp-0001
274/274 [==============================] - 190s 502ms/step - loss: 0.3763 - Accuracy: 0.8496 - val_loss: 2.1288 - val_Accuracy: 0.4985 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.2588 - Accuracy: 0.8964
Epoch 2: val_Accuracy improved from 0.49847 to 0.49887, saving model to Checkpoints/ParT_kt2/cp-0002
274/274 [==============================] - 134s 488ms/step - loss: 0.2588 - Accuracy: 0.8964 - val_loss: 2.4670 - val_Accuracy: 0.4989 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [===========

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.3841 - Accuracy: 0.8288
Epoch 1: val_Accuracy improved from -inf to 0.84140, saving model to Checkpoints/ParT_z2/cp-0001
274/274 [==============================] - 189s 508ms/step - loss: 0.3841 - Accuracy: 0.8288 - val_loss: 0.3677 - val_Accuracy: 0.8414 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3097 - Accuracy: 0.8708
Epoch 2: val_Accuracy did not improve from 0.84140
274/274 [==============================] - 133s 485ms/step - loss: 0.3097 - Accuracy: 0.8708 - val_loss: 0.5973 - val_Accuracy: 0.7182 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [==============================] - ETA: 0s - loss: 0.2746 - Acc

In [ ]:
num_partons = 128

ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full_1M')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain_1M')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m2_1M')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta_1M')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt_1M')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z_1M')

In [ ]:
num_partons = 128
training_data = import_top_data('train', dataset_size=250000,padding=num_partons)

batch_size = 256
sustain_ratio = 0.7
epochs = 8

ParT_Full.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)

Importing data...

Data imported.

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.2191 - Accuracy: 0.9111
Epoch 1: val_Accuracy improved from -inf to 0.91816, saving model to Checkpoints/ParT_Full_1M/cp-0001
684/684 [==============================] - 391s 496ms/step - loss: 0.2191 - Accuracy: 0.9111 - val_loss: 0.2038 - val_Accuracy: 0.9182 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.1914 - Accuracy: 0.9221
Epoch 2: val_Accuracy improved from 0.91816 to 0.92768, saving model to Checkpoints/ParT_Full_1M/cp-0002
684/684 [==============================] - 334s 489ms/step - loss: 0.1914 - Accuracy: 0.9221 - val_loss: 0.1814 - val_Accuracy: 0.9277 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate 

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3685 - Accuracy: 0.8501
Epoch 1: val_Accuracy improved from -inf to 0.83947, saving model to Checkpoints/ParT_Plain_1M/cp-0001
684/684 [==============================] - 229s 267ms/step - loss: 0.3685 - Accuracy: 0.8501 - val_loss: 0.4020 - val_Accuracy: 0.8395 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2559 - Accuracy: 0.8950
Epoch 2: val_Accuracy improved from 0.83947 to 0.91116, saving model to Checkpoints/ParT_Plain_1M/cp-0002
684/684 [==============================] - 180s 263ms/step - loss: 0.2559 - Accuracy: 0.8950 - val_loss: 0.2139 - val_Accuracy: 0.9112 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [====

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.2988 - Accuracy: 0.8805
Epoch 1: val_Accuracy improved from -inf to 0.90561, saving model to Checkpoints/ParT_m2_1M/cp-0001
684/684 [==============================] - 390s 494ms/step - loss: 0.2988 - Accuracy: 0.8805 - val_loss: 0.2258 - val_Accuracy: 0.9056 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2043 - Accuracy: 0.9173
Epoch 2: val_Accuracy improved from 0.90561 to 0.91572, saving model to Checkpoints/ParT_m2_1M/cp-0002
684/684 [==============================] - 334s 489ms/step - loss: 0.2043 - Accuracy: 0.9173 - val_loss: 0.2027 - val_Accuracy: 0.9157 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [==========

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3243 - Accuracy: 0.8648
Epoch 1: val_Accuracy improved from -inf to 0.78301, saving model to Checkpoints/ParT_delta_1M/cp-0001
684/684 [==============================] - 386s 489ms/step - loss: 0.3243 - Accuracy: 0.8648 - val_loss: 0.5224 - val_Accuracy: 0.7830 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2158 - Accuracy: 0.9112
Epoch 2: val_Accuracy improved from 0.78301 to 0.91655, saving model to Checkpoints/ParT_delta_1M/cp-0002
684/684 [==============================] - 330s 482ms/step - loss: 0.2158 - Accuracy: 0.9112 - val_loss: 0.2006 - val_Accuracy: 0.9165 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [====

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.2952 - Accuracy: 0.8813
Epoch 1: val_Accuracy improved from -inf to 0.50136, saving model to Checkpoints/ParT_kt_1M/cp-0001
684/684 [==============================] - 386s 488ms/step - loss: 0.2952 - Accuracy: 0.8813 - val_loss: 1.9191 - val_Accuracy: 0.5014 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2109 - Accuracy: 0.9139
Epoch 2: val_Accuracy improved from 0.50136 to 0.88760, saving model to Checkpoints/ParT_kt_1M/cp-0002
684/684 [==============================] - 331s 483ms/step - loss: 0.2109 - Accuracy: 0.9139 - val_loss: 0.2573 - val_Accuracy: 0.8876 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [==========

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3377 - Accuracy: 0.8571
Epoch 1: val_Accuracy improved from -inf to 0.88385, saving model to Checkpoints/ParT_z_1M/cp-0001
684/684 [==============================] - 385s 490ms/step - loss: 0.3377 - Accuracy: 0.8571 - val_loss: 0.2841 - val_Accuracy: 0.8839 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2531 - Accuracy: 0.8962
Epoch 2: val_Accuracy improved from 0.88385 to 0.91369, saving model to Checkpoints/ParT_z_1M/cp-0002
684/684 [==============================] - 331s 484ms/step - loss: 0.2531 - Accuracy: 0.8962 - val_loss: 0.2105 - val_Accuracy: 0.9137 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [============

In [ ]:
num_partons = 128

ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full_250k')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain_250k')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m2_250k')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta_250k')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt_250k')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z_250k')


batch_size = 256
sustain_ratio = 0.7
epochs = 8

ParT_Full.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3184 - Accuracy: 0.8760
Epoch 1: val_Accuracy improved from -inf to 0.91911, saving model to Checkpoints/ParT_Full_250k/cp-0001
684/684 [==============================] - 391s 493ms/step - loss: 0.3184 - Accuracy: 0.8760 - val_loss: 0.1990 - val_Accuracy: 0.9191 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.1985 - Accuracy: 0.9193
Epoch 2: val_Accuracy did not improve from 0.91911
684/684 [==============================] - 332s 486ms/step - loss: 0.1985 - Accuracy: 0.9193 - val_loss: 0.2051 - val_Accuracy: 0.9167 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [==============================] - ETA: 0s - loss: 0.1886 -

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3479 - Accuracy: 0.8554
Epoch 1: val_Accuracy improved from -inf to 0.87007, saving model to Checkpoints/ParT_Plain_250k/cp-0001
684/684 [==============================] - 229s 268ms/step - loss: 0.3479 - Accuracy: 0.8554 - val_loss: 0.3181 - val_Accuracy: 0.8701 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2531 - Accuracy: 0.8955
Epoch 2: val_Accuracy improved from 0.87007 to 0.91471, saving model to Checkpoints/ParT_Plain_250k/cp-0002
684/684 [==============================] - 186s 272ms/step - loss: 0.2531 - Accuracy: 0.8955 - val_loss: 0.2083 - val_Accuracy: 0.9147 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3129 - Accuracy: 0.8764
Epoch 1: val_Accuracy improved from -inf to 0.88640, saving model to Checkpoints/ParT_m2_250k/cp-0001
684/684 [==============================] - 392s 497ms/step - loss: 0.3129 - Accuracy: 0.8764 - val_loss: 0.2566 - val_Accuracy: 0.8864 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2016 - Accuracy: 0.9182
Epoch 2: val_Accuracy improved from 0.88640 to 0.92528, saving model to Checkpoints/ParT_m2_250k/cp-0002
684/684 [==============================] - 336s 491ms/step - loss: 0.2016 - Accuracy: 0.9182 - val_loss: 0.1847 - val_Accuracy: 0.9253 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [======

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3187 - Accuracy: 0.8683
Epoch 1: val_Accuracy improved from -inf to 0.91027, saving model to Checkpoints/ParT_delta_250k/cp-0001
684/684 [==============================] - 389s 492ms/step - loss: 0.3187 - Accuracy: 0.8683 - val_loss: 0.2176 - val_Accuracy: 0.9103 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2155 - Accuracy: 0.9113
Epoch 2: val_Accuracy improved from 0.91027 to 0.92099, saving model to Checkpoints/ParT_delta_250k/cp-0002
684/684 [==============================] - 332s 485ms/step - loss: 0.2155 - Accuracy: 0.9113 - val_loss: 0.1926 - val_Accuracy: 0.9210 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3117 - Accuracy: 0.8778
Epoch 1: val_Accuracy improved from -inf to 0.51355, saving model to Checkpoints/ParT_kt_250k/cp-0001
684/684 [==============================] - 396s 501ms/step - loss: 0.3117 - Accuracy: 0.8778 - val_loss: 1.4854 - val_Accuracy: 0.5135 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2205 - Accuracy: 0.9102
Epoch 2: val_Accuracy improved from 0.51355 to 0.74945, saving model to Checkpoints/ParT_kt_250k/cp-0002
684/684 [==============================] - 339s 496ms/step - loss: 0.2205 - Accuracy: 0.9102 - val_loss: 0.5217 - val_Accuracy: 0.7495 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [======

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3835 - Accuracy: 0.8498
Epoch 1: val_Accuracy improved from -inf to 0.87684, saving model to Checkpoints/ParT_z_250k/cp-0001
684/684 [==============================] - 389s 493ms/step - loss: 0.3835 - Accuracy: 0.8498 - val_loss: 0.2931 - val_Accuracy: 0.8768 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2576 - Accuracy: 0.8950
Epoch 2: val_Accuracy improved from 0.87684 to 0.91157, saving model to Checkpoints/ParT_z_250k/cp-0002
684/684 [==============================] - 333s 486ms/step - loss: 0.2576 - Accuracy: 0.8950 - val_loss: 0.2158 - val_Accuracy: 0.9116 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [========

In [ ]:
num_partons = 128

ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full_250k2')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain_250k2')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m2_250k2')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta_250k2')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt_250k2')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z_250k2')


batch_size = 256
sustain_ratio = 0.7
epochs = 8

ParT_Full.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=epochs, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=sustain_ratio, early_stopping=False)

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.2823 - Accuracy: 0.8866
Epoch 1: val_Accuracy improved from -inf to 0.90909, saving model to Checkpoints/ParT_Full_250k2/cp-0001
684/684 [==============================] - 396s 494ms/step - loss: 0.2823 - Accuracy: 0.8866 - val_loss: 0.2161 - val_Accuracy: 0.9091 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2064 - Accuracy: 0.9155
Epoch 2: val_Accuracy improved from 0.90909 to 0.92107, saving model to Checkpoints/ParT_Full_250k2/cp-0002
684/684 [==============================] - 335s 491ms/step - loss: 0.2064 - Accuracy: 0.9155 - val_loss: 0.1934 - val_Accuracy: 0.9211 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684 [

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3988 - Accuracy: 0.8342
Epoch 1: val_Accuracy improved from -inf to 0.88113, saving model to Checkpoints/ParT_Plain_250k2/cp-0001
684/684 [==============================] - 232s 268ms/step - loss: 0.3988 - Accuracy: 0.8342 - val_loss: 0.2900 - val_Accuracy: 0.8811 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
684/684 [==============================] - ETA: 0s - loss: 0.2573 - Accuracy: 0.8947
Epoch 2: val_Accuracy improved from 0.88113 to 0.90791, saving model to Checkpoints/ParT_Plain_250k2/cp-0002
684/684 [==============================] - 180s 264ms/step - loss: 0.2573 - Accuracy: 0.8947 - val_loss: 0.2222 - val_Accuracy: 0.9079 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/8
684/684

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/8
684/684 [==============================] - ETA: 0s - loss: 0.3042 - Accuracy: 0.8817
Epoch 1: val_Accuracy improved from -inf to 0.52685, saving model to Checkpoints/ParT_m2_250k2/cp-0001
684/684 [==============================] - 390s 490ms/step - loss: 0.3042 - Accuracy: 0.8817 - val_loss: 1.5052 - val_Accuracy: 0.5269 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/8
314/684 [============>.................] - ETA: 2:37 - loss: 0.2124 - Accuracy: 0.9139

## Old results: ParT has since been updated

In [ ]:
ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full2')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain2')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m22')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta2')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt2')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z2')

batch_size = 256

ParT_Full.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.8, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.8, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.8, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.8, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.8, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.8, early_stopping=False)

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.4703 - Accuracy: 0.7999
Epoch 1: val_Accuracy improved from -inf to 0.82437, saving model to Checkpoints/ParT_Full/cp-0001
274/274 [==============================] - 301s 885ms/step - loss: 0.4703 - Accuracy: 0.7999 - val_loss: 0.4125 - val_Accuracy: 0.8244 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3104 - Accuracy: 0.8699
Epoch 2: val_Accuracy did not improve from 0.82437
274/274 [==============================] - 238s 868ms/step - loss: 0.3104 - Accuracy: 0.8699 - val_loss: 0.4142 - val_Accuracy: 0.8177 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [==============================] - ETA: 0s - loss: 0.2779 - A

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.4561 - Accuracy: 0.8076
Epoch 1: val_Accuracy improved from -inf to 0.83443, saving model to Checkpoints/ParT_Plain/cp-0001
274/274 [==============================] - 124s 278ms/step - loss: 0.4561 - Accuracy: 0.8076 - val_loss: 0.3802 - val_Accuracy: 0.8344 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3109 - Accuracy: 0.8708
Epoch 2: val_Accuracy improved from 0.83443 to 0.85007, saving model to Checkpoints/ParT_Plain/cp-0002
274/274 [==============================] - 73s 265ms/step - loss: 0.3109 - Accuracy: 0.8708 - val_loss: 0.3437 - val_Accuracy: 0.8501 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [========

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.5049 - Accuracy: 0.7841
Epoch 1: val_Accuracy improved from -inf to 0.84517, saving model to Checkpoints/ParT_m2/cp-0001
274/274 [==============================] - 288s 844ms/step - loss: 0.5049 - Accuracy: 0.7841 - val_loss: 0.3647 - val_Accuracy: 0.8452 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3222 - Accuracy: 0.8640
Epoch 2: val_Accuracy improved from 0.84517 to 0.85410, saving model to Checkpoints/ParT_m2/cp-0002
274/274 [==============================] - 226s 825ms/step - loss: 0.3222 - Accuracy: 0.8640 - val_loss: 0.3395 - val_Accuracy: 0.8541 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [=============

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.5385 - Accuracy: 0.7831
Epoch 1: val_Accuracy improved from -inf to 0.82240, saving model to Checkpoints/ParT_delta/cp-0001
274/274 [==============================] - 294s 865ms/step - loss: 0.5385 - Accuracy: 0.7831 - val_loss: 0.4348 - val_Accuracy: 0.8224 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3164 - Accuracy: 0.8669
Epoch 2: val_Accuracy improved from 0.82240 to 0.85423, saving model to Checkpoints/ParT_delta/cp-0002
274/274 [==============================] - 233s 851ms/step - loss: 0.3164 - Accuracy: 0.8669 - val_loss: 0.3454 - val_Accuracy: 0.8542 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [=======

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.4804 - Accuracy: 0.8031
Epoch 1: val_Accuracy improved from -inf to 0.75793, saving model to Checkpoints/ParT_kt/cp-0001
274/274 [==============================] - 289s 853ms/step - loss: 0.4804 - Accuracy: 0.8031 - val_loss: 0.6041 - val_Accuracy: 0.7579 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3216 - Accuracy: 0.8640
Epoch 2: val_Accuracy improved from 0.75793 to 0.85663, saving model to Checkpoints/ParT_kt/cp-0002
274/274 [==============================] - 227s 831ms/step - loss: 0.3216 - Accuracy: 0.8640 - val_loss: 0.3369 - val_Accuracy: 0.8566 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [=============

Loading data...

Loaded data.
Compiling model...

Model compiled.
Beginning training... 


Epoch 1: LearningRateScheduler setting learning rate to 0.001.
Epoch 1/20
274/274 [==============================] - ETA: 0s - loss: 0.5847 - Accuracy: 0.7773
Epoch 1: val_Accuracy improved from -inf to 0.80900, saving model to Checkpoints/ParT_z/cp-0001
274/274 [==============================] - 293s 871ms/step - loss: 0.5847 - Accuracy: 0.7773 - val_loss: 0.4095 - val_Accuracy: 0.8090 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.001.
Epoch 2/20
274/274 [==============================] - ETA: 0s - loss: 0.3163 - Accuracy: 0.8667
Epoch 2: val_Accuracy did not improve from 0.80900
274/274 [==============================] - 233s 852ms/step - loss: 0.3163 - Accuracy: 0.8667 - val_loss: 0.7626 - val_Accuracy: 0.6919 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.001.
Epoch 3/20
274/274 [==============================] - ETA: 0s - loss: 0.2834 - Accu

In [ ]:
import numpy as np
import awkward
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')
def stack_arrays(a, keys, axis=-1):
    flat_arr = np.stack([a[k].flatten() for k in keys], axis=axis)
    return awkward.JaggedArray.fromcounts(a[keys[0]].counts, flat_arr)
def pad_array(a, maxlen, value=0., dtype='float32'):
    x = (np.ones((len(a), maxlen)) * value).astype(dtype)
    for idx, s in enumerate(a):
        if not len(s):
            continue
        trunc = s[:maxlen].astype(dtype)
        x[idx, :len(trunc)] = trunc
    return x

class Dataset(object):

    def __init__(self, filepath, feature_dict = {}, dataset_size = -1, label='label', pad_len=100, data_format='channel_first'):
        self.filepath = filepath
        self.feature_dict = feature_dict
        if len(feature_dict)==0:
            feature_dict['points'] = ['part_etarel', 'part_phirel']
            feature_dict['features'] = ['part_pt_log', 'part_e_log', 'part_etarel', 'part_phirel']
            feature_dict['mask'] = ['part_pt_log']
        self.label = label
        self.pad_len = pad_len
        assert data_format in ('channel_first', 'channel_last')
        self.stack_axis = 1 if data_format=='channel_first' else -1
        self._values = {}
        self._label = None
        self._load(dataset_size)

    def _load(self, dataset_size):
        logging.info('Start loading file %s' % self.filepath)
        counts = None
        with awkward.load(self.filepath) as a:
            self._label = a[self.label][:dataset_size]
            for k in self.feature_dict:
                cols = self.feature_dict[k]
                if not isinstance(cols, (list, tuple)):
                    cols = [cols]
                arrs = []
                for col in cols:
                    if counts is None:
                        counts = a[col].counts
                    else:
                        assert np.array_equal(counts, a[col].counts)
                    arrs.append(pad_array(a[col], self.pad_len)[:dataset_size])
                self._values[k] = np.stack(arrs, axis=self.stack_axis)
        logging.info('Finished loading file %s' % self.filepath)


    def __len__(self):
        return len(self._label)

    def __getitem__(self, key):
        if key==self.label:
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]

train_dataset = Dataset('converted/train_file_0.awkd', dataset_size = 70000,data_format='channel_last')
val_dataset = Dataset('converted/val_file_0.awkd', dataset_size = 30000,data_format='channel_last')

import tensorflow as tf
from tensorflow import keras
import tf_keras_model
from tf_keras_model import get_particle_net, get_particle_net_lite

model_type = 'particle_net' # choose between 'particle_net' and 'particle_net_lite'
num_classes = train_dataset.y.shape[1]
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

  # Training parameters
#batch_size = 1024 if 'lite' in model_type else 384

epochs = 20

def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['Accuracy'])

 # Prepare model model saving directory.
import os
save_dir = 'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_Accuracy',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

train_dataset.shuffle()
model.fit(train_dataset.X, train_dataset.y,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=epochs, # --- train only for 1 epoch here for demonstration ---
          validation_data=(val_dataset.X, val_dataset.y),
          shuffle=True,
          verbose=1,
          callbacks=callbacks)

In [ ]:
ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full2')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain2')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m22')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta2')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt2')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z2')

batch_size = 256

ParT_Full.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)

In [ ]:
ParT_Full = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_Full3')
ParT_Plain = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt', 'z', 'delta', 'm2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], isPlain=True, model_name='ParT_Plain3')

ParT_m2 = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['m2'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_m23')

ParT_delta = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['delta'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_delta3')

ParT_kt = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['kt'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_kt3')

ParT_z = ParticleTransformerWrapper(d_model=[128, 512, 128], d_pair_model=[64,64,64], 
                                        max_number_partons=num_partons, input_dim=11, 
                                        pairwise_outputs=['z'], num_heads=8, N_p=8, N_c=2, output_dims=[[1,0]], model_name='ParT_z3')

batch_size = 256

ParT_Full.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_Plain.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_m2.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_delta.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_kt.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
ParT_z.train(dataset=training_data, epochs=20, train_val_split=0.3, batch_size=batch_size, dynamic_lr=True, max_lr=0.001, sustain_ratio=0.85, early_stopping=False)
